![](https://savingexpert.org/wp-content/uploads/2018/11/prosper-personal-loans-reviews.png)

<h2 align = 'center'>FACTORS THAT AFFECT PROSPER RATING AND HOW IT CAN BE USED TO PREDICT DELINQUENCY</h2>

## Updated: September 13, 2022

## Investigation Overview

In this investigation, I look at the features of prosper loan that could be used to predict borrower's delinquency. The main focus was on the prosper rating, monthly income, occupation, and employment status. The analysis was divide into two stages, the [exploratory](Exploratory_Analysis.ipynb), partially covered, and explanatory analysis which is the scope of this work.

## Dataset Overview

This data set includes customers who have paid off their loans, who have been past due and put into collection without paying back their loan and interests, and who have paid off only after they were put in collection. The original dataset contains 113937 rows and 81 columns out of which 12 features of intrest were selected.Eight hundred  and seventy-one data points were removed from the analysis due to inconsistencies or missing information.

In [1]:
# import all packages and set plots to be embedded inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [3]:
# Load in your dataset and describe its properties

p_loan = pd.read_csv('../input/prosper-loan-dataset/prosperLoanData.csv',
                     usecols = ['LoanKey', 'Term', 'LoanStatus', 'LoanOriginalAmount', 'ListingCategory (numeric)',
                                'ProsperRating (Alpha)', 'ListingCreationDate', 'BorrowerRate', 'IsBorrowerHomeowner', 'Occupation',
                                'StatedMonthlyIncome', 'EmploymentStatus', 'DelinquenciesLast7Years'], low_memory = True)

p_loan = p_loan.reindex(columns=['LoanKey', 'Term', 'LoanStatus', 'LoanOriginalAmount', 'ListingCategory (numeric)',
                                 'ProsperRating (Alpha)', 'ListingCreationDate', 'BorrowerRate', 'IsBorrowerHomeowner',
                                 'Occupation', 'StatedMonthlyIncome', 'EmploymentStatus', 'DelinquenciesLast7Years'])
print(p_loan.shape)
p_loan.head()

## Filter out unique values

In [4]:
## Check to be sure that LoanKey has no dulicate value(s)
p_loan.LoanKey.duplicated().sum()

In [5]:
## Drop duplicates to get unique LoanKey
p_loan.drop_duplicates(subset = 'LoanKey', inplace = True)
p_loan.LoanKey.duplicated().sum()

In [6]:
p_loan.info()

In [7]:
p_loan.isnull().sum()

In [8]:
## Rename LoanOriginalAmount, ListingCategory (numeric), ListingCreationDate, IsBorrowerHomeowner, and StatedMonthlyIncome 
## as LoanAmount, ListingCategory, ListingnDate, Homeowner, and MonthlyIncome respectively

p_loan.rename(columns = {
    'LoanOriginalAmount': 'LoanAmount',
    'ListingCategory (numeric)': 'ListingCategory',
    'ProsperRating (Alpha)': 'ProsperRating',
    'ListingCreationDate': 'ListingDate',
    'IsBorrowerHomeowner': 'Homeowner',
    'StatedMonthlyIncome': 'MonthlyIncome'}, inplace = True)

# conver changes
p_loan.head()

## Clean data

### Make the values of the ListingCategory more readable

In [9]:
# Mapping each number with what it represents
number_label = {0:'Not Availbale', 1:'Debt Consolidation', 2:'Home Improvement', 3:'Business', 4:'Personal Loan',
                5:'Student', 6:'Auto', 7:'Other', 8:'Baby&Adoption', 9:'Boat', 10:'Cosmetic Procedure', 
                11: 'Engagement Ring', 12:'Green Loans', 13:'Household Expenses', 14:'Large Purchases',
                15:'Medical/Dental', 16:'Motorcycle', 17:'RV', 18:'Taxes', 19:'Vacation', 20:'Wedding Loans'}

# Function to apply
def category_label(p_loan):
    if p_loan['ListingCategory'] in number_label.keys():
        label = number_label[p_loan['ListingCategory']]
        return label
    else:
        return p_loan['ListingCategory']

# Apply function
p_loan['ListingCategory'] = p_loan.apply(category_label, axis=1)

# Confirm changes
p_loan.head()

###  convert ProsperRating (Alpha) into ordered categorical types

In [10]:
p2_loan = p_loan.copy()
p2_loan.loc[p2_loan['ProsperRating'].isnull(), 'ProsperRating'] = 'yes'
p3_loan = p2_loan.copy()
p3_loan.info()

In [11]:
ordinal_var = ['AA', 'A', 'B', 'C', 'D', 'E', 'HR', 'NILL']
ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = ordinal_var)
p3_loan['ProsperRating'] = p3_loan['ProsperRating'].astype(ordered_var)

#confirm changes
p3_loan.info()

In [12]:
p2_loan.ProsperRating.unique()

In [13]:
ordinal_var = ['AA', 'A', 'B', 'C', 'D', 'E', 'HR']
ordered_var = pd.api.types.CategoricalDtype(ordered = True, categories = ordinal_var)
p_loan['ProsperRating'] = p_loan['ProsperRating'].astype(ordered_var)

#confirm changes
p_loan.info()

### Drop empty values
- In DelinquenciesLast7Years features since the percentage of the missing values is small
- In ProsperRating being a key feature in this analysis

In [14]:
## Calculate the percentage of missing values in DelinquenciesLast7Years
print(f'{p_loan.DelinquenciesLast7Years.isnull().sum() / len(p_loan.DelinquenciesLast7Years) *100}%')

p_loan.isnull().sum()

In [15]:
## Extract only the rows with no-missing values in DelinquenciesLast7Years and ProsperRating
p_clean = p_loan[p_loan['DelinquenciesLast7Years'].notnull() & p_loan['ProsperRating'].notnull()]

# Confirm changes
p_clean.isnull().sum()

### Replace missing values in *`Occupation`*  with *`Not Available`* to be human readable

In [16]:
p_clean['Occupation'] = p_clean['Occupation'].fillna('Not Available')

# Confirm change
p_clean.isnull().sum()

### Format Datatypes

In [17]:
## Change ListingnDate to datetime
p_clean['ListingDate'] = p_clean['ListingDate'].str.extract('(\d{4}[-]\d{2}[-]\d{2})', expand = True)
p_clean['ListingDate'] = pd.to_datetime(p_clean['ListingDate'])

# Change DelinquenciesLast7Years to integer
p_clean['DelinquenciesLast7Years'] = p_clean['DelinquenciesLast7Years'].astype(int)

## Confirm changes
p_clean.info()

In [18]:
p_loan.describe()

In [19]:
p_clean.head()

## Distribution of Delinquency

Delinquencies in the last seven years has a long-tailed distribution, [exploratory analysis](), with a lot of borrowers at the low end of delinqueces, and few on the high end. When plotted on a log-scale, the delinquences distribution has roughly one peak around 10 and decreases to the left of the plot.

In [49]:
# Ivestigating futher on a bigger binsize
log_binsize = 0.04
bins_max = np.log10(p_clean['DelinquenciesLast7Years'].max())
bins = 10 ** np.arange(0.9, bins_max + log_binsize, log_binsize)

plt.figure(figsize = [10, 8])
plt.hist(data = p_clean, x = 'DelinquenciesLast7Years', bins = bins)
plt.xscale('log')
plt.x_ticks = [2, 10, 20, 50, 100]
plt.xlabel('Delinquencies', size = 15)
plt.ylabel('Count', size = 15)
plt.title('Delinquencies In The Last Seven Years', size = 25)
plt.show();

In [24]:
# These are functions defined to plot Bar Graphs of Univariate plots using Seaborn's Countplot function.

def bar_texts(var, x, **kwargs):
    prosper = var.value_counts()
    base_color = sb.color_palette()[0]
    ax = plt.figure(figsize= (10, 8)).subplots()
    sb.countplot(data= p_clean, x= x, y= None, hue= None, color= base_color);
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    locs, labels  = plt.xticks()

    for loc, label in zip(locs, labels):
        count = prosper[label.get_text()]
        percentage = '{:.2f}%'.format((count * 100) / prosper.sum())
        m = plt.text(loc, count, percentage, ha= 'center', va= 'baseline')
    return m

def barh_texts(var, y, **kwargs):
    prosper = var.value_counts()
    base_color = sb.color_palette()[0]
    ax = plt.figure(figsize= (10, 8)).subplots()
    sb.countplot(data= p_clean, y= y, color= base_color);
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    locs, labels = plt.yticks()
    
    for loc, label in zip(locs, labels):
        count = prosper[label.get_text()]
        percentage = '{:.2f}%'.format((count * 100) / prosper.sum())
        m = plt.text(count, loc, percentage, va= 'center', ha= 'left')

## Distribution of Prosper Rating

Looking at the plot below, we can see that a larger number of Borrowers has an Alpha Prosper Rating of A, B, C and D (17.13%, 18.30%, 21.55%, 16.87%) these translate to 2.00–3.99%, 4.00–5.99%, 6.00–8.99%, 9.00–11.99% Estimated Average Annual Loss Rat respectively. The largest value, 21.55%, is observed at C Rating. This implies that investor, on average stands a risk of not bein paid back by 4% - 8% which a good risk to take

In [39]:
# A bar plot showing borrowers prosper ratings.
bar_texts(p_clean['ProsperRating'], 'ProsperRating')
plt.title('Bar plot of Alpha Prosper Ratings Count', size = 25)
plt.xlabel("Prosper Rating", size = 15)
plt.ylabel("Count", size = 15);

## Correlation Between Numerical Variables

While most of the varibles are negatively correlated with delinquencies, Borrower's rate, however, is possively correlated with delinquencies wich is also much more expected. 
This implies that as these variables increase, the probability that the borrower won't pay back decreases and vice versa

In [27]:
numeric_vars = ['LoanAmount', 'MonthlyIncome', 'DelinquenciesLast7Years', 'BorrowerRate', 'Term']
categoric_vars = ['EmploymentStatus', 'ProsperRating', 'ListingCategory', 'Homeowner', 'LoanStatus']

In [28]:
# select low outliers, using criteria eyeballed from the plots
low_outliers = p_clean['MonthlyIncome'] < 0
high_outliers = p_clean['MonthlyIncome'] > 1 *1e5
print(f'low_outliers: {low_outliers.sum()}\nhigh_outliers: {high_outliers.sum()}')
p_clean = p_clean.loc[-high_outliers]
p_clean.loc[high_outliers,:]

In [37]:
# correlation plot
plt.figure(figsize = [12,9])
sb.heatmap(p_clean[numeric_vars].corr(), annot = True, fmt = '.3f',
           cmap = 'vlag_r', center = 0)
plt.title("Heatmap Showing Corraletion Between Numerical Variables", size = 20)
plt.show()

## Prosper Rating and House Ownnership

The relationship between house ownership of the borrower and the prosper rating. Observing the first four high ratings, AA A, B, and C, majority of the borrowers are house owners while the reverse is for low ratings, D and E.

In [35]:
# A plot showing the relation between Prosper Rating and Home owner variables.
plt.figure(figsize = [12,9])
sb.countplot(data= p_clean, x= 'ProsperRating', hue= 'Homeowner', palette = 'Blues')
plt.xlabel("Prosper Rating", size = 15)
plt.ylabel("Count", size = 15)
plt.title('Relationship Between Prosper Rating and Home ownership', size = 20);

## Delinquencies, Monthly Income and Prosper Rating

Here, it can be seen that the prosper rating AA is commonly associated with high monthly income and low delinqueny. The reverse is seen for rating E. While rating B is somehow evenly distributed.

In [40]:
# A plot showing the relationship between Delinquencies, Monthly Income and Prosper Rating
cat_markers = p_clean.ProsperRating.unique()
plt.figure(figsize = [12,9])
for cat in cat_markers:
    clean_cat = p_clean[p_clean.ProsperRating == cat]
    plt.scatter(data = clean_cat, x = 'DelinquenciesLast7Years', y = 'MonthlyIncome', alpha = 0.5)

plt.xlabel("Delinquencies in the Last Seven Years", size = 15)
plt.ylabel("Monthly Income", size = 15)
plt.title('Delinquencies, Monthly Income and Prosper Rating', size = 20)
plt.legend(cat_markers, title= 'Prosper Rating');

## Delinquencies, Borrower Rate and Prosper Rating

With this plot, I can save conclude that borrowers with good prosper rating, AA, borrow less frequently and rearly default. The opposite is true for those with bad or poor rating, E and HR. The same logic hold for other ratings as well

In [41]:
# A plot showing the relationship between Delinquencies, Borrower Rate and Prosper Rating
cat_markers = p_clean.ProsperRating.unique()
plt.figure(figsize = [12,9])
for cat in cat_markers:
    clean_cat = p_clean[p_clean.ProsperRating == cat]
    plt.scatter(data = clean_cat, x = 'DelinquenciesLast7Years', y = 'BorrowerRate', alpha = 0.5)

plt.xlabel('Delinquencies in the Last Seven Years', size = 15)
plt.ylabel('Borrower Rate', size = 15)
plt.title('Delinquencies, Borrower Rate and Prosper Rating', size = 20)
plt.legend(cat_markers, title= 'Prosper Rating');

# Findings
- Borrowers has an Alpha Prosper Rating of A, B, C and D (17.13%, 18.30%, 21.55%, 16.87%) these translate to 2.00–3.99%, 4.00–5.99%, 6.00–8.99%, 9.00–11.99% Estimated Average Annual Loss Rat respectively. The largest value, 21.55%, is observed at C Rating. This implies that investor, on average stands a risk of not bein paid back by 4% - 8% which a good risk to take

- Most of the varibles are negatively correlated with delinquencies, Borrower's rate, however, is possively correlated with delinquencies wich is also much more expected. This implies that as these variables increase, the probability that the borrower won't pay back decreases and vice versa

- Observing the first four high ratings, AA A, B, and C, majority of the borrowers are house owners while the reverse is for low ratings, D and E.

- Prosper rating AA is commonly associated with high monthly income and low delinqueny. The reverse is seen for rating E. While rating B is somehow evenly distributed.

- Borrowers with good prosper rating, AA, borrow less frequently and rearly default. The opposite is true for those with bad or poor rating, E and HR. The same logic hold for other ratings as well

- Looking at this relatonship, it is logic to think that because this people earn good income, they don't usually borrow. Perhaps they do when they plan to carryout an important project such as buuilding a house and they are able to pay back when they are done with such a project. Less wonder while this kind of borrower always demand for long term loans.


<a id = 'conclusion'></a>

# Conclusions
From the insights, I can conclude that the monthly income, loan amount and borrower rate are factors that greatly affect the prosper rating which is a good predictor of delinquency.


### Uncomment line of code below to produce a powerpoint presentation

In [ ]:
!jupyter nbconvert Explanatory_Analysis.ipynb --to slides --post serve --no-input --no-prompt